In [1]:
!pip install pydub numpy noisereduce soundfile librosa
!apt install ffmpeg
!apt-get install libsndfile1

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libsndfile1 is already the newest version (1.0.31-2ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.


## 1. Разделение аудио на чанки


In [ ]:
from pydub import AudioSegment
from pydub.utils import make_chunks
import os

In [ ]:
def split_audio(input_file, output_folder, count_chunks, format='mp3'):
    audio = AudioSegment.from_file(input_file)
    chunk_length = len(audio) // count_chunks
    chunks = make_chunks(audio, chunk_length)

    output_files = []
    for i, chunk in enumerate(chunks):
        output_file_name = f'chunk_{i}.{format}'
        output_path = os.path.join(output_folder, output_file_name)
        chunk.export(output_path, format)
        output_files.append(output_path)

    return output_files

In [ ]:
input_file = ''
output_folder = ''
count_chunks = 10

split_audio(input_file, output_folder, count_chunks)

## 2. Фильтрация шума на аудио

In [ ]:
import numpy as np
from pydub import AudioSegment
import noisereduce
import soundfile
import librosa
import os

In [ ]:
def filter_sound(input_file, output_folder, noise_start=0, noise_duration=0.5, reduction_strength=0.8, format='mp3'):
    temp_input_file = 'temp_input_file.wav'
    temp_filter_file = 'temp_filter_file.wav'

    try:
        audio = AudioSegment.from_file(input_file)
        audio.export(temp_input_file, format='wav')

        y, sr = librosa.load(temp_input_file, sr=None)
        noise_samples = y[int(noise_start * sr):int((noise_start + noise_duration) * sr)]
        filtered_y = noisereduce.reduce_noise(
            y=y,
            sr=sr,
            y_noise=noise_samples,
            stationary=True,
            prop_decrease=reduction_strength
        )

        soundfile.write(temp_filter_file, filtered_y, sr)

        filtered_audio = AudioSegment.from_wav(temp_filter_file)
        file_name = f'filtered_audio.{format}'
        output_path = os.path.join(output_folder, file_name)
        filtered_audio.export(output_path, format)
    finally:
        if os.path.exists(temp_input_file):
            os.remove(temp_input_file)
        if os.path.exists(temp_filter_file):
            os.remove(temp_filter_file)

In [ ]:
input_file = '/content/drive/MyDrive/DHDHDBDHDHS.mp3'
output_folder = '/content/sample_data'

filter_sound(input_file, output_folder)

## 3. Нормализация звука

In [ ]:
from pydub import AudioSegment, effects
import os

In [ ]:
def normalize_audio(input_file, output_folder, format='mp3'):
    audio = AudioSegment.from_file(input_file)
    normalized_audio = effects.normalize(audio)

    file_name = f'normalized_audio.{format}'
    output_path = os.path.join(output_folder, file_name)
    normalized_audio.export(output_path, format)

    return output_path

In [ ]:
input_file = ''
output_folder = ''

normalize_audio(input_file, output_folder)

'/content/sample_data/normalized_audio.mp3'

## 4. Полная обработка аудио

In [2]:
from pydub import AudioSegment, effects
from pydub.utils import make_chunks

import noisereduce
import soundfile
import librosa
import numpy as np
import os

Далее я перерабатываю методы для использования в одной функции

In [3]:
def normalize_audio(input_file):
    audio = AudioSegment.from_file(input_file)
    normalized_audio = effects.normalize(audio)

    return normalized_audio

In [4]:
def filter_sound(audio, noise_start=0, noise_duration=0.5, reduction_strength=0.8):
    temp_input_file = 'temp_input_file.wav'
    temp_filter_file = 'temp_filter_file.wav'

    try:
        audio.export(temp_input_file, format='wav')

        y, sr = librosa.load(temp_input_file, sr=None)
        noise_samples = y[int(noise_start * sr):int((noise_start + noise_duration) * sr)]
        filtered_y = noisereduce.reduce_noise(
            y=y,
            sr=sr,
            y_noise=noise_samples,
            stationary=True,
            prop_decrease=reduction_strength
        )

        soundfile.write(temp_filter_file, filtered_y, sr)
        filtered_audio = AudioSegment.from_wav(temp_filter_file)
    finally:
        if os.path.exists(temp_input_file):
            os.remove(temp_input_file)
        if os.path.exists(temp_filter_file):
            os.remove(temp_filter_file)

    return filtered_audio

In [5]:
def split_audio(audio, output_folder, count_chunks, format='mp3'):
    chunk_length = len(audio) // count_chunks
    chunks = make_chunks(audio, chunk_length)

    output_files = []
    for i, chunk in enumerate(chunks):
        output_file_name = f'chunk_{i}.{format}'
        output_path = os.path.join(output_folder, output_file_name)
        chunk.export(output_path, format)
        output_files.append(output_path)

    return output_files

In [6]:
def process_audio(input_file, output_folder, count_chunks, format='mp3'):
    norm_audio = normalize_audio(input_file)
    filtered_audio = filter_sound(norm_audio)
    output_files = split_audio(filtered_audio, output_folder, count_chunks, format)

    return output_files

In [7]:
input_file = ''
output_folder = ''
count_chunks = 3

process_audio(input_file, output_folder, count_chunks)

['/content/sample_data/chunk_0.mp3',
 '/content/sample_data/chunk_1.mp3',
 '/content/sample_data/chunk_2.mp3']